<a href="https://colab.research.google.com/github/ImNotDanish05/2025_KC_TI-1A/blob/main/P11_Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import os
print(tf.__version__)

2.18.0


In [9]:
"""
List Dir: https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images
Open File: https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images
"""

base_dir = 'https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images'
train_dir = base_dir + '/train'
validation_dir = base_dir + '/val'

In [50]:
import requests
from urllib.parse import urlparse
def listdirx(dirx):
  response = requests.get(dirx)
  if response.status_code == 200:
      files = response.json()
      print(dirx)
      for file in files:
          print(f"- {file['name']}")
  else:
      print(f"Gagal mengambil data. Status: {response.status_code}")
def list_all_png_from_github_api(api_url, collected=[]):
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"❌ Gagal mengakses {api_url} | Status: {response.status_code}")
        return collected

    items = response.json()
    for item in items:
        if item['type'] == 'file' and item['name'].endswith('.png'):
            collected.append(item['download_url'])
        elif item['type'] == 'dir':
            list_all_png_from_github_api(item['url'], collected)
    return collected
def download_images(image_urls, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for url in image_urls:
        filename = os.path.basename(urlparse(url).path)
        save_path = os.path.join(save_dir, filename)

        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ Downloaded: {filename}")
        else:
            print(f"❌ Gagal download: {filename}")

In [24]:
listdirx(train_dir)
listdirx(validation_dir)

https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/train
- .DS_Store
- clean
- messy
https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/val
- clean
- messy


TensorFlow

Documentation: https://keras.io/api/data_loading/image/

In [67]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [68]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(input_shape=(150,150,3), include_top=False, weights='imagenet')
base_model.trainable = False  # Jangan dulu di-train ulang, cukup frozen
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

<ipython-input-68-2521272334>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150,150,3), include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
train_clean_urls = list_all_png_from_github_api(train_dir + "/clean")
train_messy_urls = list_all_png_from_github_api(train_dir + "/messy")
validation_clean_urls = list_all_png_from_github_api(validation_dir + "/clean")
validation_messy_urls = list_all_png_from_github_api(validation_dir + "/messy")
download_images(train_clean_urls, 'dataset/train/clean')
download_images(train_messy_urls, 'dataset/train/messy')
download_images(validation_clean_urls, 'dataset/val/clean')
download_images(validation_messy_urls, 'dataset/val/messy')

In [69]:
print("Clean:", len(os.listdir('/content/dataset/train/clean')))
print("Messy:", len(os.listdir('/content/dataset/train/messy')))
print("Clean:", len(os.listdir('/content/dataset/val/clean')))
print("Messy:", len(os.listdir('/content/dataset/val/messy')))

Clean: 96
Messy: 96
Clean: 96
Messy: 96


In [66]:
train_generator = train_datagen.flow_from_directory(
        "/content/dataset/train",  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        "/content/dataset/val", # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')
print(train_generator.class_indices)

Found 192 images belonging to 2 classes.
Found 192 images belonging to 2 classes.
{'clean': 0, 'messy': 1}


In [59]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'), # 512 / 64
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
model.summary() # Untuk melihat ringkasan model

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 15, 15, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,529,665 (51.61 MB)

 Trainable params: 13,529,665 (51.61 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [63]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    verbose=2
)

Epoch 1/30
48/48 - 37s - 779ms/step - accuracy: 0.4583 - loss: 0.7728 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/30
48/48 - 41s - 860ms/step - accuracy: 0.5000 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/30
48/48 - 32s - 667ms/step - accuracy: 0.5000 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/30
48/48 - 34s - 699ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/30
48/48 - 33s - 694ms/step - accuracy: 0.5000 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 6/30
48/48 - 32s - 671ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 7/30


KeyboardInterrupt: 

In [39]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

test_image = "https://i.pinimg.com/736x/31/af/bb/31afbb9580927635c0a076f99bb696fb.jpg"
download_images([test_image], 'dataset/test')

def predict_image_class(model, image_path):
    img = Image.open(image_path).resize((150, 150))
    imgplot = plt.imshow(img)
    x = np.expand_dims(np.array(img), axis=0)
    classes = model.predict(x, batch_size=10)

    if classes[0] == 0:
        print('clean')
    else:
        print('messy')

# Misalkan model adalah objek model yang sudah diinisialisasi sebelumnya

# Mengunggah gambar
image_path = "/content/dataset/test/31afbb9580927635c0a076f99bb696fb.jpg"
predict_image_class(model, image_path)

✅ Downloaded: 31afbb9580927635c0a076f99bb696fb.jpg
